In [75]:
from pygromos.files.topology import top
from pygromos.files.topology import ptp
from pygromos.files.coord import cnf

import copy
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem

In [48]:
in_top1_path = "F313.top"
in_top2_path = "G078.top"
out_ptp_path = "test.ptp"

in_coord2_path = "G078_unitedatom_optimised_geometry.g96"
in_coord1_path = "F313_unitedatom_optimised_geometry.g96"

## COORD

In [49]:
cmol1 = cnf.Cnf(in_coord1_path)
cmol2 = cnf.Cnf(in_coord2_path)

In [50]:
cmol2.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [51]:
 cmol1.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Atom Matching

In [52]:
from rdkit.Chem import rdFMCS

def find_atom_mapping(cnfA:cnf.Cnf, cnfB:cnf.Cnf)->(dict,str):
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)


    #CompareElements
    mcs = rdFMCS.FindMCS([mol1, mol2],
                         atomCompare=rdFMCS.AtomCompare.CompareAnyHeavyAtom,
                         bondCompare=rdFMCS.BondCompare.CompareOrderExact,
                         ringMatchesRingOnly=True, completeRingsOnly=True)
    #                    ringCompare=rdFMCS.RingCompare.StrictRingFusion)
    smartsString = mcs.smartsString

    ##MCS Match
    patt = Chem.MolFromSmarts(smartsString)  # smartsString
    mol1Match = mol1.GetSubstructMatch(patt)
    mol2Match = mol2.GetSubstructMatch(patt)

    atom_mappingAB = dict(zip(mol1Match, mol2Match))
    return atom_mappingAB, smartsString

atom_mappingAB, smart = find_atom_mapping(cnfA=cmol1, cnfB=cmol2)
len(atom_mappingAB), atom_mappingAB, smart

(10,
 {2: 1, 1: 0, 3: 9, 5: 8, 7: 6, 11: 4, 13: 2, 14: 3, 12: 5, 8: 7},
 '[#6&R]1(-&!@[#8,#6;!R])-&@[#6&R]-&@[#6&R]-&@[#6&R](-&@[#6&R](-&@[#6&R]-&@1-&!@[#1&!R])-&!@[#1&!R])-&!@[#7,#6;!R]')

## COORDINATE Generation
### Coord Alignment

In [53]:
def align_cnfs_with_MCS(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atom_mappingAB:int)->(cnf.Cnf, cnf.Cnf):
    
    cnfA= copy.deepcopy(cnfA)
    cnfB= copy.deepcopy(cnfB)
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)
    
    AllChem.AlignMol(mol1, mol2, atomMap=list(atom_mappingAB.items()))


    conf2 = mol2.GetConformer()
    for ind, pos in enumerate(conf2.GetPositions()):
        cnfB.POSITION[ind].xp= pos[0]/10
        cnfB.POSITION[ind].yp= pos[1]/10
        cnfB.POSITION[ind].zp= pos[2]/10

    conf1 = mol1.GetConformer()
    for ind, pos in enumerate(conf1.GetPositions()):
        cnfA.POSITION[ind].xp= pos[0]/10
        cnfA.POSITION[ind].yp= pos[1]/10
        cnfA.POSITION[ind].zp= pos[2]/10
        
    return cnfA, cnfB

cmol1, cmol2 = align_cnfs_with_MCS(cnfA=cmol1, cnfB=cmol2, atom_mappingAB=atom_mappingAB)



### State merging

In [109]:
len(cmol1.POSITION), len(cmol2.POSITION)

(15, 18)

In [112]:
eucldean_dist = lambda x,y: np.sqrt((x.xp-y.xp)**2+(x.yp-y.yp)**2+(x.zp-y.zp)**2)

def merge_states(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atomMatchingAB:dict, dist_tresh:float=0.09,_doNotChangeAtomType:bool=False)-> cnf.Cnf:#nm
    
    
    natoms_in_a=len(cnfA.POSITION)
    match_molA, match_molB = np.array(list(atom_mappingAB.items())).T

    cmol_comb = copy.deepcopy(cnfA)    
    
    for pos in cnfB.POSITION:
        #print(pos)
        #print(pos.atomID in selected_b_match)
        #print(any([eucldean_dist(pos, pos2) < dist_tresh for pos2 in cmol_comb.POSITION]), [eucldean_dist(pos, pos2) <dist_tresh for pos2 in cmol_comb.POSITION])
        #print([eucldean_dist(pos, pos2)  for pos2 in selected_b_cnf.POSITION])

        if(pos.atomID in match_molB or 
           any([eucldean_dist(pos, pos2) < dist_tresh for pos2 in cnfA.POSITION])):#check distance
            continue
        else:
            #print('new')
            pos = copy.deepcopy(pos)
            pos.resID=2
            pos.atomID += natoms_in_a
            cmol_comb.POSITION.append(pos)

    #clean cmol_comb
    present_atoms = []
    for ind, pos in enumerate(cmol_comb.POSITION):
        pos.resID=1
        if(not _doNotChangeAtomType): pos.atomType = "C"
        pos.atomID = ind+1
        pos.resName=cmol1.POSITION[0].resName[:2]+cmol2.POSITION[0].resName[2:4]
        present_atoms.append(pos.atomID)
    return cmol_comb

cmol_comb = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09)
cmol_comb.write("fung.g96")
cmol_comb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [113]:
cmol_comb.POSITION

POSITION
# 	 
    1 F378  C          1   -0.213727417    0.354335876   -0.003240671
    1 F378  C          2   -0.093019160    0.279001792    0.000243936
    1 F378  C          3   -0.102939305    0.141918380    0.000711725
    1 F378  C          4    0.018250526    0.071394519    0.000004694
    1 F378  C          5    0.111304909    0.127475534   -0.000850611
    1 F378  C          6    0.019387384   -0.067653748   -0.001712013
    1 F378  C          7    0.114627096   -0.120425292   -0.002634784
    1 F378  C          8   -0.100731452   -0.141029430   -0.002129426
    1 F378  C          9   -0.099284927   -0.281761402    0.003451760
    1 F378  C         10   -0.182330588   -0.324460779   -0.036355167
    1 F378  C         11   -0.016782612   -0.323047736   -0.038974489
    1 F378  C         12   -0.221332678   -0.069621200   -0.001120014
    1 F378  C         13   -0.315639720   -0.123849744   -0.000763904
    1 F378  C         14   -0.223083517    0.070253457    0.000796961
    1 

## Topology

In [14]:
top1 = top.Top(in_top1_path)
top2 = top.Top(in_top2_path)


In [15]:
com_top = top2
com_top += top1
red_com_top = copy.deepcopy(com_top)

In [56]:
#BONDS
bonds =[]
for bond in com_top.BOND:
    if(bond.IB in present_atoms and bond.JB in present_atoms):
        bonds.append(bond)
red_com_top.BOND.content = bonds
red_com_top.BOND.NBON = len(bonds)

#BondAngles
angles =[]
for angle in com_top.BONDANGLE:
    if(angle.IT in present_atoms and angle.JT in present_atoms and angle.KT in present_atoms):
        angles.append(angle)

red_com_top.BONDANGLE.content = angles
red_com_top.BONDANGLE.NTHE = len(angles)

#BondAnglesH
anglesH =[]
for angleH in com_top.BONDANGLEH:
    if(angleH.IT in present_atoms and angleH.JT in present_atoms and angleH.KT in present_atoms):
        anglesH.append(angle)

red_com_top.BONDANGLEH.content = anglesH
red_com_top.BONDANGLEH.NTHEH = len(anglesH)

#Dihedrals
dihedrals =[]
for dih in com_top.DIHEDRAL:
    if(dih.IP in present_atoms and dih.JP in present_atoms and dih.KP in present_atoms and dih.LP in present_atoms):
        dihedrals.append(dih)

red_com_top.DIHEDRAL.content = dihedrals
red_com_top.DIHEDRAL.NPHI = len(dihedrals)

#DihedralsH
dihedralsH =[]
for dihh in com_top.DIHEDRALH:
    if(dihh.IPH in present_atoms and dihh.JPH in present_atoms and dihh.KPH in present_atoms and dihh.LPH in present_atoms):
        dihedralsH.append(dihh)

red_com_top.DIHEDRALH.content = dihedralsH
red_com_top.DIHEDRALH.NPHIH = len(dihedralsH)

#Impropers
impdihedrals =[]
for imp in com_top.IMPDIHEDRAL:
    if(imp.IQ in present_atoms and imp.JQ in present_atoms and imp.KQ in present_atoms and imp.LQ in present_atoms):
        impdihedrals.append(dihh)

red_com_top.IMPDIHEDRAL.content = impdihedrals
red_com_top.IMPDIHEDRAL.NQHI = len(impdihedrals)

#ImpropersH
impdihedralsH =[]
for impH in com_top.IMPDIHEDRALH:
    if(impH.IQH in present_atoms and impH.JQH in present_atoms and impH.KQH in present_atoms and impH.LQH in present_atoms):
        impdihedralsH.append(dihh)

red_com_top.IMPDIHEDRALH.content = impdihedralsH
red_com_top.IMPDIHEDRALH.NQHIH = len(impdihedralsH)

#Mass, VDW, Charge
sol =[]
for atm in com_top.SOLUTEATOM:
    if(atm.ATNM in present_atoms):
        sol.append(atm)

red_com_top.SOLUTEATOM.content = sol 
red_com_top.SOLUTEATOM.NRP = len(sol)

"""
#crossDIHEDRAL
crds =[]
for crd in com_top.CROSSDIHEDRAL:
    print(vars(crd))
    if(crd.ATNM in present_atoms):
        crds.append(atm)

red_com_top.CROSSDIHEDRAL.content = crds 

#crossDIHEDRALH
crdsH =[]
for crdH in com_top.CROSSDIHEDRALH:
    print(vars(crdH))
    if(crdH.ATNM in present_atoms):
        crdsH.append(atm)

red_com_top.CROSSDIHEDRALH.content = crdsH
"""

##resname meshing
molA_Name = com_top.RESNAME.content[1][0]
molB_Name = com_top.RESNAME.content[2][0]

red_com_top.RESNAME.content = [['1'], [molA_Name[:2]+molB_Name[:2]]]


pass

### Build PTP

In [21]:
ptp_file = ptp.Ptp()

In [17]:
from pygromos.files.blocks.pertubation_blocks import PERTPROPERDIH, atom_lam_pertubation_state_dihedral

In [23]:
pertproperdih =PERTPROPERDIH()

In [24]:
for 
pertproperdih.content.append()

SyntaxError: invalid syntax (Temp/ipykernel_7584/1245646628.py, line 1)

## DONE

#### Dual Topology Approach

In [114]:
def generate_dual_topology_approach(cnfA, cnfB, topA, topB, eds:bool=False):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb = copy.deepcopy(cnfA)
    #cnf_comb += cnfB # needs to be implemented
    
    for pos in cnfB.POSITION:
        cnf_comb.POSITION.append(pos)
    
    cnf_comb.supress_atomPosition_singulrarities()


    ##Top
    nAtoms_top1 = len(top1.SOLUTEATOM)
    top_comb = copy.deepcopy(top1)
    top_comb += top2
    
    ### Pertubation
    ptp_comb = ptp.Pertubation_topology()
    
    if(eds):
        from pygromos.files.blocks.pertubation_blocks import MPERTATOM
        from pygromos.files.blocks.pertubation_blocks import  atom_eds_pertubation_state, pertubation_eds_state

        tops = [topA, topB]
        dummyState = pertubation_eds_state(IAC=22, CHARGE=0)

        numStates=len(tops)
        IND = 1
        atom_states = []
        for top_ind, top in enumerate(tops):
            for atom in top.SOLUTEATOM:        
                states = {}
                for ctop in range(ntops):
                    if(ctop==top_ind):
                        states.update({ctop+1:pertubation_eds_state(IAC=atom.IAC, CHARGE=atom.CG)})
                    else:
                        states.update({ctop+1:dummyState})

                atom_ptp = atom_eds_pertubation_state(NR=IND, NAME=atom.PANM, STATES=states)
                atom_states.append(atom_ptp)
                IND+=1

        ptp_comb.add_block(block=MPERTATOM(NJLA=len(atom_states), NPTB=numStates, STATEATOMS=atom_states))

    else:
        from pygromos.files.blocks.pertubation_blocks import PERTATOMPARAM
        from pygromos.files.blocks.pertubation_blocks import  atom_lam_pertubation_state, pertubation_lam_state_nonbonded

        tops = [topA, topB]
        build_dummyState = lambda m: pertubation_lam_state_nonbonded(IAC=22, CHARGE=0, MASS=m)

        numStates=len(tops)
        IND = 1
        atom_states = []
        for top_ind, top in enumerate(tops):
            for atom in top.SOLUTEATOM:        
                states = {}
                for ctop in range(ntops):
                    if(ctop==top_ind):
                        states.update({ctop+1:pertubation_lam_state_nonbonded(IAC=atom.IAC, CHARGE=atom.CG, MASS=atom.MASS)})
                    else:
                        states.update({ctop+1:build_dummyState(atom.MASS)})

                atom_ptp = atom_lam_pertubation_state(NR=IND, RES=atom.MRES, NAME=atom.PANM, STATES=states,)
                atom_states.append(atom_ptp)
                IND+=1

        ptp_comb.add_block(block=PERTATOMPARAM(NJLA=len(atom_states), STATEATOMS=atom_states))
    
    return cnf_comb, top_comb, ptp_comb

In [115]:
cnf_file, top_file, ptp_file = generate_dual_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [116]:
cnf_file.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [117]:
top_file.SOLUTEATOM

SOLUTEATOM
#   NRP: number of solute atoms
 	 33
#  ATNM: atom number
#  MRES: residue number
#  PANM: atom name of solute atom
#   IAC: integer (van der Waals) atom type code
#  MASS: mass of solute atom
#    CG: charge of solute atom
#   CGC: charge group code (0 or 1)
#   INE: number of excluded atoms
# INE14: number of 1-4 interactions
# ATNM MRES PANM IAC     MASS       CG  CGC INE
#                                           INE14
	1	1	C5	16	15.035	0.275	1	2	2	3
										2	4	14
	2	1	O1	4	15.9994	-0.4	1	3	3	4	14
										4	5	6	12	15
	3	1	C4	64	12.011	0.269	1	7	4	5	6	8	12	14
											15
										2	7	13
	4	1	C3	64	12.011	-0.214	1	6	5	6	7	8	12	14
										2	9	15
	5	1	H4	20	1.008	0.144	1	1	6
										3	7	8	14
	6	1	C2	64	12.011	-0.334	1	5	7	8	9	12	14
										3	10	11	13
	7	1	H3	20	1.008	0.186	1	1	8
										2	9	12
	8	1	C1	64	12.011	0.462	1	6	9	10	11	12	13	14
										1	15
	9	1	N1	66	14.0067	-0.95	1	3	10	11	12
										2	13	14
	10	1	H1	59	1.008	0.39	1	1	11
										1	12
	11	1

In [118]:
ptp_file

PERTATOMPARAM
# NJLA  	 NPTB =  	 2 	 
 	 33
# state_identifiers
#  	 state1 	 state2
#    NR   RES  NAME  IAC1mass1   CHARGE1  IAC2mass2   CHARGE2    ALPHLJ     ALPHCRF   
    1     1    C5    16 15.035    0.27500    22 15.035    0.00000    1.00000    1.00000
    2     1    O1     4 15.9994   -0.40000    22 15.9994    0.00000    1.00000    1.00000
    3     1    C4    64 12.011    0.26900    22 12.011    0.00000    1.00000    1.00000
    4     1    C3    64 12.011   -0.21400    22 12.011    0.00000    1.00000    1.00000
    5     1    H4    20 1.008    0.14400    22 1.008    0.00000    1.00000    1.00000
    6     1    C2    64 12.011   -0.33400    22 12.011    0.00000    1.00000    1.00000
    7     1    H3    20 1.008    0.18600    22 1.008    0.00000    1.00000    1.00000
    8     1    C1    64 12.011    0.46200    22 12.011    0.00000    1.00000    1.00000
    9     1    N1    66 14.0067   -0.95000    22 14.0067    0.00000    1.00000    1.00000
   10     1    H1    59 1.008    0.

#### Hybrid Topology Approach

In [119]:
def generate_hybrid_topology_approach(cnfA, cnfB, topA, topB):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, _doNotChangeAtomType=True, dist_tresh=0.0) #no distance collapsing
    
    ##Top
    
    
    ### Pertubation
    
    
    return cnf_comb
    #return cnf_comb, top_comb, ptp_comb

In [120]:
cnf_file= generate_hybrid_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [121]:
cnf_file.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [122]:
cnf_file

TITLE
	 >>> Generated with python lib function_libs utilities. (riniker group)	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'	 >>> Generated with python lib function_libs utilities. (riniker group)	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'
	 >>> Generated with python lib function_libs utilities. (riniker group)
	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'
END
POSITION
# 	 
    1 F378  C5         1   -0.213727417    0.354335876   -0.003240671
    1 F378  O1         2   -0.093019160    0.279001792    0.000243936
    1 F378  C4         3   -0.102939305    0.141918380    0.000711725
    1 F378  C3         4    0.018250526    0.071394519    0.000004694
    1 F378  H4         5    0.111304909    0.127475534   -0.000850611
    1 F378  C2         6    0.019387384   -0.067653748   -0.001712013
    1 F378  H3         7    0.114627096   -0.120425292   -0.002634784
    1 F378  C1         8   -0.100731452   -0.141029430   -0.

#### Single Topology Approach

In [123]:
def generate_single_topology_approach(cnfA, cnfB, topA, topB):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09) #no distance collapsing
    

    return cnf_comb #, top_comb, ptp_comb

In [124]:
cnf_file= generate_single_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [125]:
cnf_file.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [126]:
cnf_file

TITLE
	 >>> Generated with python lib function_libs utilities. (riniker group)	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'	 >>> Generated with python lib function_libs utilities. (riniker group)	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'
	 >>> Generated with python lib function_libs utilities. (riniker group)
	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'
END
POSITION
# 	 
    1 F378  C          1   -0.213727417    0.354335876   -0.003240671
    1 F378  C          2   -0.093019160    0.279001792    0.000243936
    1 F378  C          3   -0.102939305    0.141918380    0.000711725
    1 F378  C          4    0.018250526    0.071394519    0.000004694
    1 F378  C          5    0.111304909    0.127475534   -0.000850611
    1 F378  C          6    0.019387384   -0.067653748   -0.001712013
    1 F378  C          7    0.114627096   -0.120425292   -0.002634784
    1 F378  C          8   -0.100731452   -0.141029430   -0.